# 음성인식 메모프로그램

Pyside를 이용하여 지금까지 작성했던 메모 프로그램에 기존 강좌에서 다룬 음성인식 기능을 활용하여 메모작성시 키보드로만 입력하지 않고 음성을 통해 이를 메모할 수 있는 음성인식 + 메모 프로그램을 제작해보도록 합니다. 음성인식 기능과 메모 프로그램을 결합하기 위해선 PySide에서의 쓰레드 개념에 대해서 알고 넘어가야 합니다.

### GUI 프로그램에서 문제점

In [ ]:
import sys
from PySide2 import QtWidgets, QtCore
import time

class MyWindow(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("남박사의 파이썬")
        self.button = QtWidgets.QPushButton(self)
        self.textbox = QtWidgets.QTextEdit(self)

        self.button.setText("동작테스트")
        self.button.clicked.connect(self.buttonClick)

        layout = QtWidgets.QVBoxLayout()
        layout.addWidget(self.button)
        layout.addWidget(self.textbox)

        self.resize(400, 200)
        self.setLayout(layout)
        self.show()

    def buttonClick(self):
        print("버튼 클릭")
        while True:
            str_time = "{}".format(time.time())
            self.textbox.append(str_time)
            print(str_time)
            time.sleep(0.5)

app = QtWidgets.QApplication(sys.argv)
win = MyWindow()
sys.exit(app.exec_())

PySide 를 이용한 GUI 프로그램에서는 위와 같은 코드에서 문제가 발생합니다. 위의 코드에서처럼 버튼을 클릭했을시에 프로그램 내의 어떤 동작을 CPU가 처리하느라 다른 일을 하지 못하는 상황으로 파이썬 뿐만 아니라 기본적으로 단일 쓰레드로 동작하는 모든 프로그램에 생기는 현상입니다. 위의 코드를 실행해보면 ```print(str_time)``` 구문에 의해 프롬프트 창에 시간이 출력은 되지만 실제 GUI 프로그램 자체는 응답없음이 발생됩니다. 이런것 처럼 위의 코드에서 버튼을 클릭했을 경우 프로그램이 ```while True``` 문을 수행하느라 다른 GUI 구성요소를 그리는 작업을 할 수 없는 상황으로 GUI 프로그램에서는 특히나 더 중요한 부분입니다. 그래서 일반적인 GUI 프로그램에서는 연산 부분과 GUI 처리 부분을 서로 분리되게끔 설계 하는게 중요합니다.

### PySide 에서의 쓰레드
파이썬에서는 쓰레드를 사용하기 Threading, concurrent 등의 객체가 있고 이를 사용해도 되지만 PySide 의 GUI 환경에서는 클래스 내의 자원에 접근하거나 이벤트등을 관리하게 되는 경우가 많은데 이를 위해 QThread 라는 객체를 사용하는게 좋습니다.

In [ ]:
import sys
from PySide2 import QtWidgets, QtCore
import time

class MyThread(QtCore.QThread):
    def __init__(self, parent=None):
        super().__init__()
        self.isRun = False
        self.parent = parent

    def run(self):
        while self.isRun:
            str_time = "{}".format(time.time())
            self.parent.textbox.append("{}".format(time.time()))
            print(str_time)
            self.sleep(0.5)

class MyWindowThread(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("남박사의 파이썬")
        self.button = QtWidgets.QPushButton(self)
        self.textbox = QtWidgets.QTextEdit(self)

        self.button.setText("동작테스트")
        self.button.clicked.connect(self.buttonClick)

        layout = QtWidgets.QVBoxLayout()
        layout.addWidget(self.button)
        layout.addWidget(self.textbox)

        self.th = MyThread(self)

        self.resize(400, 200)
        self.setLayout(layout)
        self.show()

    def buttonClick(self):
        if not self.th.isRun:
            print('메인 : 쓰레드 시작')
            self.th.isRun = True
            self.th.start()

if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    win = MyWindowThread()
    sys.exit(app.exec_())

PySide에서 QThread 를 사용하기 위해서는 위의 코드에서 처럼 QThread 를 상속받은 MyThread 클래스를 구현하여 동작을 분리합니다. ```def __init___(self, parent=None)``` 의 생성자 함수에서는 ```parent=None``` 인자를 받는데 부모쪽에서 해당 클래스의 인스턴트를 생성시에 ```self.th = MyThread(self)``` 로 넘겨준 ```self``` 값이 ```parent``` 에 들어가게 되며 추후 부모의 자원에 접근하는데 사용됩니다. QThread 의 ```run()``` 함수는 오버라이딩 된 함수입니다. 부모쪽에서 버튼 클릭시에 ```self.th.start()``` 함수를 호출하게 되면 QThread 의 ```run()``` 이 수행되게 됩니다.


### QThread 에서의 시그널과 슬롯

쓰레드는 프로그램의 메모리 영역안에 귀속되지만 독립적으로 동작하며 QWidget 과 같은 GUI 클래스들은 내부적으로 쓰레드가 구현되어있습니다. 그렇기 때문에 위 코드에서처럼 ```QThread``` 를 상속받아 만든 ```MyThread``` 에서 부모의 자원으로 직접 접근해서 사용하는 방법은 절대 권장되지 않는 방법입니다. 이렇게 쓰레드간 자원을 공유하거나 어떤 결과를 전달할때는 시그널(이벤트)과 슬롯(이벤트핸들러)을 사용해서 전달합니다.

In [ ]:
import sys
from PySide2 import QtWidgets, QtCore
import time

class MyThread(QtCore.QThread):
    # 이벤트를 발생시킬 쓰레드 내의 시그널 함수
    # 인자값을 넘길시에 반드시 자료형을 작성해야 합니다.
    signal_thread = QtCore.Signal(str)

    def __init__(self, parent=None):
        super().__init__()
        self.isRun = False
        self.parent = parent

    def run(self):
        while self.isRun:
            str_time = "{}".format(time.time())
            # 쓰레드에서 시그널을 발생(emit)
            self.signal_thread.emit(str_time)
            print(str_time)
            self.sleep(1)


class MyWindowThread(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("남박사의 파이썬")
        self.button = QtWidgets.QPushButton(self)
        self.textbox = QtWidgets.QTextEdit(self)

        self.button.setText("동작테스트")
        self.button.clicked.connect(self.buttonClick)

        layout = QtWidgets.QVBoxLayout()
        layout.addWidget(self.button)
        layout.addWidget(self.textbox)

        self.th = MyThread(self)
        # 쓰레드에 슬롯 함수 연결
        self.th.signal_thread.connect(self.slot_thread)

        self.resize(400, 200)
        self.setLayout(layout)
        self.show()

    def slot_thread(self, text):
        '''쓰레드에서 넘어온 시그널을 처리할 슬롯 함수'''
        self.textbox.append(text)

    def buttonClick(self):
        if not self.th.isRun:
            print('메인 : 쓰레드 시작')
            self.th.isRun = True
            self.th.start()


if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    win = MyWindowThread()
    sys.exit(app.exec_())

<pre style="background-color:#eeeeee">
class MyThread(QtCore.QThread):
    signal_thread = QtCore.Signal(str)
</pre>
MyThread 클래스에 보면 위 처럼 ```signal_thread = QtCore.Signal()``` 을 선언하는데 이 변수는 외부의 MyWindowThread 클래스를 보면 ```self.th.signal_thread.connect(self.slot_thread)``` 이렇게 ```slot_thread()``` 슬롯 함수와 연결 되게 됩니다. 이렇게 연결된 시그널과 슬롯을 통해 ```QThread``` 를 상속받아 쓰레드로 동작하는 ```MyThread``` 클래스에서 부모인 ```MyWindowsThread``` 클래스에 어떤 이벤트 발생을 알리거나 데이터를 전송할 수 있게 됩니다.

### 음성 인식 명령

우리가 작성할 음성인식 메모 프로그램은 오디오 입력은 항상 대기 상태로 무한루프로 동작하기 때문에 GUI 프로그램에서 이를 쓰레드로 분리하지 않고 구현하면 메인 프로그램이 응답없음 상태가 되게 됩니다. 그렇기 때문에 반드시 쓰레드를 구현하여 작성해야 합니다.

In [ ]:
import sys
from PySide2 import QtCore, QtGui, QtWidgets
import os
import sqlite3 as sql
import datetime
import speech_recognition as sr

def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source)
        said = ""
        try:
            said = r.recognize_google(audio, language="ko-KR")
            print("오디오인식: {}".format(said))
        except sr.UnknownValueError:
            print("구글 인식 불가")
        except sr.RequestError as e:
            print("구글 오류; {0}".format(e))
        except sr.WaitTimeoutError as e:
            print("타임 아웃 {0}".format(e))
    return said

class Worker(QtCore.QThread):
    sig_make_memo = QtCore.Signal()
    sig_update_memo = QtCore.Signal(str)
    sig_delete_memo = QtCore.Signal()
    sig_close_memo = QtCore.Signal()
    sig_exit_thread = QtCore.Signal()

    def __init__(self, parent=None):
        super().__init__()
        self.parent = parent

    def run(self):
        while True:
            txt = get_audio().strip()

            print("Listen... {}".format(txt))
            if txt == "메모":
                self.sig_make_memo.emit()
                while True:
                    txt = get_audio()
                    if txt == "저장":
                        break
                    self.sig_update_memo.emit(txt)
            elif txt == "메모 삭제":
                self.sig_delete_memo.emit()
            elif txt == "메모 닫기":
                self.sig_close_memo.emit()
            elif txt == "종료":
                break
        self.sig_exit_thread.emit()

- sig_make_memo : 새로운 메모창을 생성 합니다.
- sig_update_memo : 메모 내용을 업데이트 합니다.
- sig_delete_memo : 메모를 삭제합니다.
- sig_close_memo : 메모위젯을 닫습니다.
- sig_exit_thread : 오디오를 입력받는 메인 루프를 종료합니다.

위의 설명처럼 우리가 작성할 ```QThread``` 를 상속받아 만든 ```Worker``` 클래스에는 5개의 시그널이 있습니다. ```run()``` 함수에서는 ```while True``` 무한루프를 돌며 ```get_audio()``` 함수를 통해 오디오를 입력받습니다. 어디서부터 어디까지가 "메모"의 영역으로 들어갈지를 생각해봐야 하는데 위의 코드에서처럼 입력받은 오디오에서 "메모" 라는 내용이 있으면 ```sig_makke_memo``` 시그널을 발생시켜 새로운 메모창을 생성하고 또 ```while``` 문을 통해 그때부터 메모를 받아들여 부모쪽에 ```sig_update_memo``` 메모를 작성하는 시그널을 발생시켜 메모를 작성하게 합니다. 이 메모 데이터는 "저장" 이라는 내용이 나올때까지 모두 메모 내용으로 판단하게 됩니다.

그 외 "메모삭제", "메모닫기", "종료" 등의 데이터에 따라 메모를 삭제하거나 메모창을 닫거나 프로그램을 종료하는 시그널을 부모쪽으로 전송하게 됩니다.

In [ ]:
class MyApp():
    def __init__(self):
        self.app = QtWidgets.QApplication(sys.argv)
        self.windows = []
        self.cur_dir = os.path.dirname(__file__)
        self.db_name = self.cur_dir + "\\memo.db"
        self.initDB()
        self.loadDB()
        self.current_memo = None

        self.th = Worker()
        self.th.sig_make_memo.connect(self.memo_make)
        self.th.sig_update_memo.connect(self.update_memo)
        self.th.sig_delete_memo.connect(self.memo_delete)
        self.th.sig_close_memo.connect(self.memo_close)
        self.th.sig_exit_thread.connect(self.exit_thread)
        self.th.start()

    def exit_thread(self):
        sys.exit()

    def memo_close(self):
        self.current_memo.close()

    def memo_delete(self):
        self.current_memo.delete_window()

    def update_memo(self, text):
        if self.current_memo is not None:
            self.current_memo.add_memo(text)

    def memo_make(self):
        self.create_new_memo()

class MyMemo(QtWidgets.QWidget):
    #... 코드 생략
    def add_memo(self, text):
        cur_txt = self.note.toPlainText()
        text = text.replace("엔터", "\n").replace("공백", " ").replace("쉼표", ",").replace("마침표", ".").replace("느낌표", "!").replace("물음표", "?")

        if text == "라인 삭제":
            lines = str(cur_txt).split("\n")
            cur_txt = "\n".join(lines[:-1])
            text = ""
        text = text.replace(" \n ", "\n")
        cur_txt += text
        self.note.setText(cur_txt)
        self.note.moveCursor(QtGui.QTextCursor.End)

기존에 작성한 메모프로그램의 ```MyApp``` 클래스의 생성자 함수에 쓰레드를 생성하고 각 슬롯 함수를 연결시키고 쓰레드를 동작 시키는 코드를 위와 같이 추가하였습니다. ```self.current_memo``` 변수는 오디오를 통해 "메모" 를 인식해서 새로운 창을 띄웠을때 현재 새롭게 띄운 메모위젯을 대상으로만 음성인식 기능을 제공하기 위해 새롭게 생성된 메모 위젯을 기억하기 위한 변수 입니다.

실제 메모 윈도우를 구현한 ```MyMemo``` 클래스에는 ```add_memo(self, text)``` 함수가 구현되어있는데 이 함수는 위의 ```Worker``` 클래스에서 발생시킨 ```sig_update_memo``` 시그널에 의해 부모인 ```MyApp``` 클래스의 ```update_memo()``` 슬롯 함수를 호출하게 되고 이 함수를 통해 호출되어지는 함수입니다. 이 함수가 실제 현재 새롭게 생성된 메모 위젯에 내용을 작성하는 역할을 합니다. 이 ```add_memo()``` 함수에서는 넘어온 텍스트 데이터에서 몇몇 특수문자를 치환하고 특정 단어 ("라인삭제") 등에 맞는 동작을 하게 되고 현재 메모 데이터를 구해 새로운 데이터를 추가하여 다시 작성하는 내용으로 되어있습니다.

### 완성 코드

In [ ]:
'''
pip install SpeechRecognition
pip install gTTS
pip install playsound
pip install pyaudio
pip install pycaw
'''
import sys
from PySide2 import QtCore, QtGui, QtWidgets
import os
import sqlite3 as sql
import datetime
import speech_recognition as sr


def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source)
        said = ""
        try:
            said = r.recognize_google(audio, language="ko-KR")
            print("오디오인식: {}".format(said))
        except Exception as e:
            print("Error: {}".format(e))
    return said


class Worker(QtCore.QThread):
    sig_make_memo = QtCore.Signal()
    sig_update_memo = QtCore.Signal(str)
    sig_delete_memo = QtCore.Signal()
    sig_close_memo = QtCore.Signal()
    sig_exit_thread = QtCore.Signal()

    def __init__(self, parent=None):
        super().__init__()
        self.parent = parent

    def run(self):
        while True:
            txt = get_audio().strip()

            print("Listen... {}".format(txt))
            if txt == "메모":
                self.sig_make_memo.emit()
                while True:
                    txt = get_audio()
                    if txt == "저장":
                        break
                    self.sig_update_memo.emit(txt)
            elif txt == "메모 삭제":
                self.sig_delete_memo.emit()
            elif txt == "메모 닫기":
                self.sig_close_memo.emit()
            elif txt == "종료":
                break
        self.sig_exit_thread.emit()


class MyBar(QtWidgets.QWidget):
    def __init__(self, parent, bg_color):
        super(MyBar, self).__init__()
        self.parent = parent
        self.layout = QtWidgets.QHBoxLayout()
        self.layout.setMargin(0)
        self.layout.setSpacing(0)
        self.title = QtWidgets.QLabel()

        btn_size = 30

        self.btn_new = QtWidgets.QPushButton("+")
        self.btn_new.clicked.connect(self.btn_new_clicked)
        self.btn_new.setFixedSize(btn_size, btn_size)
        self.btn_new.setStyleSheet("border:0px; background-color: {};".format(parent.color_bg))

        self.btn_close = QtWidgets.QPushButton("x")
        self.btn_close.clicked.connect(self.btn_delete_clicked)
        self.btn_close.setFixedSize(btn_size, btn_size)
        self.btn_close.setStyleSheet("border:0px; background-color: {};".format(parent.color_bg))

        self.btn_color = QtWidgets.QPushButton("c")
        self.btn_color.clicked.connect(self.btn_color_clicked)
        self.btn_color.setFixedSize(btn_size, btn_size)
        self.btn_color.setStyleSheet("border:0px; background-color: {};".format(parent.color_bg))

        self.title.setFixedHeight(30)
        self.title.setAlignment(QtCore.Qt.AlignCenter)
        self.layout.addWidget(self.btn_new)
        self.layout.addWidget(self.title)
        self.layout.addWidget(self.btn_color)
        self.layout.addWidget(self.btn_close)

        self.title.setStyleSheet("""
            background-color: {};
            color: {};
        """.format(parent.color_bg, parent.color_text))

        self.setLayout(self.layout)

    def setColor(self, color):
        self.title.setStyleSheet("background-color:" + color)
        self.btn_color.setStyleSheet("border: 0px; background-color:" + color)
        self.btn_close.setStyleSheet("border: 0px; background-color:" + color)
        self.btn_new.setStyleSheet("border: 0px; background-color:" + color)

    def btn_new_clicked(self):
        self.parent.on_create()

    def btn_close_clicked(self):
        self.parent.close()

    def btn_color_clicked(self):
        self.parent.changeBgColor()

    def btn_delete_clicked(self):
        buttonReply = QtWidgets.QMessageBox.question(self, '파이메모', "현재 메모를 삭제 하시겠습니까?", QtWidgets.QMessageBox.Yes | QtWidgets.QMessageBox.No, QtWidgets.QMessageBox.No)
        if buttonReply == QtWidgets.QMessageBox.Yes:
            print('Yes clicked.')
            self.parent.delete_window()
        else:
            print('No clicked.')
            self.parent.close()


class MyMemo(QtWidgets.QWidget):
    def __init__(self, on_create, on_close, on_delete, idx=None, text=None, rect=None, color_bg=None, color_text=None):
        super().__init__()
        self.on_create = on_create
        self.on_close = on_close
        self.on_delete = on_delete
        self.text = text
        self.rect = rect
        self.deleted = False
        self.idx = idx

        self.color_text = "#000000" if color_text is None else color_text
        self.color_bg = "#dce459" if color_bg is None else color_bg

        self.initUI()
        self.setIcon()
        self.setWindowFlags(QtCore.Qt.FramelessWindowHint)

    def setIcon(self):
        appIcon = QtGui.QIcon("icon.png")
        self.setWindowIcon(appIcon)

    def initUI(self):
        self.note = QtWidgets.QTextEdit(self)
        self.pal = QtGui.QPalette()
        self.font = QtGui.QFont("맑은 고딕")
        self.font.setPointSize(15)
        self.note.setFont(self.font)
        self.others_windows = []
        color_text = "QTextEdit {border: 0; background-color: " + self.color_bg + "; color: " + self.color_text + "}"
        self.note.setStyleSheet(color_text)

        # 위에서 color_bg, color_text 가 셋팅 된 후에 실행되야 함
        self.title_bar = MyBar(self, self.color_bg)

        self.title_bar.setStyleSheet("top: -20px")
        self.layout = QtWidgets.QVBoxLayout()
        self.layout.setSpacing(0)
        self.layout.setMargin(0)
        self.layout.addWidget(self.title_bar)
        self.layout.addWidget(self.note)
        grip = QtWidgets.QSizeGrip(self)
        self.layout.addWidget(grip, 0, QtCore.Qt.AlignBottom | QtCore.Qt.AlignRight)
        self.setLayout(self.layout)

        if self.text:
            self.note.setText(self.text)
        if self.rect:
            self.setGeometry(self.rect)
        self.show()

    def resizeEvent(self, event):
        self.oldPos = None

    def closeEvent(self, event):
        if not self.deleted:
            self.on_close(self.idx, self.geometry().getRect(), self.get_current_text(), self.color_bg, self.color_text)
        else:
            self.on_delete(self.idx)

    def mousePressEvent(self, event):
        self.oldPos = event.globalPos()

    def mouseMoveEvent(self, event):
        if self.oldPos is not None:
            delta = QtCore.QPoint(event.globalPos() - self.oldPos)
            self.move(self.x() + delta.x(), self.y() + delta.y())
            self.oldPos = event.globalPos()

    def delete_window(self):
        self.on_close = self.on_delete
        self.deleted = True
        self.close()

    def get_current_text(self):
        if not self.note.toPlainText():
            return ""
        return self.note.toHtml()

    def add_memo(self, text):
        cur_txt = self.note.toPlainText()
        print(cur_txt)
        text = text.replace("엔터", "\n").replace("공백", " ").replace("쉼표", ",").replace("마침표", ".").replace("느낌표", "!").replace("물음표", "?")

        if text == "라인 삭제":
            lines = str(cur_txt).split("\n")
            cur_txt = "\n".join(lines[:-1])
            text = ""
        text = text.replace(" \n ", "\n")

        cur_txt += text
        self.note.setText(cur_txt)
        self.note.moveCursor(QtGui.QTextCursor.End)

    def changeBgColor(self):
        selectedcolor = QtWidgets.QColorDialog.getColor().name()
        self.color_bg = selectedcolor
        self.note.setStyleSheet("border:0px; background-color:" + selectedcolor)
        self.title_bar.setColor(selectedcolor)

    def changeTextColor(self):
        selectedcolor = QtWidgets.QColorDialog.getColor()
        if QtGui.QColor.isValid(selectedcolor):
            self.pal.setColor(QtGui.QPalette.Text, selectedcolor)
            self.note.setPalette(self.pal)


class MyApp():
    def __init__(self):
        self.app = QtWidgets.QApplication(sys.argv)
        self.windows = []
        self.cur_dir = os.path.dirname(__file__)
        self.db_name = "memo.db"
        self.initDB()
        self.loadDB()
        self.current_memo = None

        self.th = Worker()
        self.th.sig_make_memo.connect(self.memo_make)
        self.th.sig_update_memo.connect(self.update_memo)
        self.th.sig_delete_memo.connect(self.memo_delete)
        self.th.sig_close_memo.connect(self.memo_close)
        self.th.sig_exit_thread.connect(self.exit_thread)
        self.th.start()

    def exit_thread(self):
        sys.exit()

    def memo_close(self):
        self.current_memo.close()

    def memo_delete(self):
        self.current_memo.delete_window()

    def update_memo(self, text):
        print(text)
        if self.current_memo is not None:
            self.current_memo.add_memo(text)

    def memo_make(self):
        self.create_new_memo()

    def initDB(self):
        self.con = sql.connect(self.db_name)
        self.cursor = self.con.cursor()
        query = """
        CREATE TABLE IF NOT EXISTS memo (
            '_idx' INTEGER PRIMARY KEY AUTOINCREMENT,
            '_memo' TEXT,
            '_pubdate' TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            '_pos_x' INTEGER,
            '_pos_y' INTEGER,
            '_width' INTEGER,
            '_height' INTEGER,
            '_color_bg' VARCHAR(30),
            '_color_text' VARCHAR(30)
        )"""
        self.con.execute(query)
        self.con.close

    def loadDB(self):
        query = 'SELECT _idx, _memo, _pos_x, _pos_y, _width, _height, _color_bg, _color_text FROM memo'

        with sql.connect(self.db_name) as con:
            cur = con.cursor()
            cur.execute(query)
            memos = cur.fetchall()
            con.commit()
            cur.close()

        for m in memos:
            idx, memo, x, y, w, h, c_bg, c_text = m
            self.create_new_memo(idx, memo, QtCore.QRect(x, y, w, h), c_bg, c_text)
        return len(memos)

    def initMemo(self):
        self.create_new_memo()

    def create_new_memo(self, idx=None, text=None, rect=None, color_bg=None, color_text=None):
        w = MyMemo(self.create_new_memo, self.close_memo, self.delete_memo, idx, text, rect, color_bg=color_bg, color_text=color_text)
        w.show()
        self.current_memo = w
        self.windows.append(w)

    def close_memo(self, idx, rect, text, color_bg, color_text):
        x, y, w, h = rect
        if text == "":
            return

        now = datetime.datetime.now()
        if idx is None:
            query = '''INSERT INTO memo (_memo, _pos_x, _pos_y, _width, _height, _pubdate, _color_bg, _color_text)
                       VALUES (?, ?, ?, ?, ?, ?, ?, ?)'''
            with sql.connect(self.db_name) as con:
                cur = con.cursor()
                cur.execute(query, (text, x, y, w, h, now, color_bg, color_text))
                con.commit()
                cur.close()
        else:
            query = '''UPDATE memo
                SET _memo=?,
                _pos_x=?,
                _pos_y=?,
                _width=?,
                _height=?,
                _pubdate=?,
                _color_bg=?,
                _color_text=?
                WHERE _idx=?'''
            with sql.connect(self.db_name) as con:
                cur = con.cursor()
                cur.execute(query, (text, x, y, w, h, now, color_bg, color_text, idx))
                con.commit()
                cur.close()

    def delete_memo(self, idx=None):
        if idx is not None:
            query = "DELETE FROM memo WHERE _idx=?"
            with sql.connect(self.db_name) as con:
                cur = con.cursor()
                cur.execute(query, (idx, ))
                con.commit()
                cur.close()

if __name__ == '__main__':
    main = MyApp()
    sys.exit(main.app.exec_())
